In [74]:
import pandas as pd
import re
import csv

In [67]:
qq = "2016-11-02 09:38:53, 76941, 72045, 18.35, Hi Megan, I wanted to make sure this is you. Before I paid you for the sweat 👕 "

In [70]:
rr = qq.split(', ')

In [73]:
ss = ""

for r in rr[4:]:
    ss+=r 
    
print(ss)

Hi MeganI wanted to make sure this is you. Before I paid you for the sweat 👕 


In [81]:
input_dir = 'paymo_input/'
batch_file = 'batch_payment.csv'
stream_file = 'stream_payment.csv'
batch_path = input_dir + batch_file
stream_path = input_dir + stream_file

test_file = 'batch_test_2.csv'
test_path = input_dir + test_file

In [100]:
with open(test_path) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        print(row[' message'])        

 🦄 
 Pitcher 
 🚕 
 For your wife. 
 🍺🍺🍺🏌🏌🏌 
 🚗🌃 
 Best hair stylist on the west coast... And rent 
 Juice 
 Milk 
 ✨ 
 Zombies 
 💋💋💋 
 🌚🌚🌚🌚 
 Debauchery in the form of adulting 
 Garlic rosemary challah 
 🚕 
 🍗 
 Delish 
 Exterminating 
 🚕 
 🚕 💸 
 🎂🎉 
 Turkey bacon avacadoooooo 
 🐱Disappoint plushie 
 😈 
 Mid-afternoon burger 
 Mimosas!!! Yaaassss queen!! 🍾 
 Fappiano🍾 
 🐶 
 . 
 Cover 
 Drank at Josh Ritter whom I ❤ 
 Slow service 
 🍕 
 GOODS 
 Drink 
 For our nightlights 
 🍦 
 Because you made waffles. 
 🚙 
 🍟 
 Lisp 
 🐚. 
 🌴🌴 
 Beaaaaan 
 Tip 
 Alc 
 🍻 
 Venmo 
 Whip whip 
 Lylas 
 😘❤️🍻 
 Polish adventures? 
 Thank you !! 
 💵🔙  refund cuz movie is in Westwood! 
 There is no potsticksr emoji 😕 
 Wheel chair 
 Nashlorette 
 2 washes 🚘🚿 
 🌚 
 Bus to logroño and burgos 
 Keep yo 💰 
 Davison ball 
 🍆 
 🚕 
 Broadway 
 New bus 
 Midgets 
 Drinks 
 🍳 
 ☕️🍔 
 🔥 
 For the wifey 
 All of King Midas' silver 
 Hustlin 💁🏾 
 Salad tossing lessons 
 Das bier 
 Uber input 
 How big were the balls? 
 

In [89]:
for row in reader:
    print(row.keys())

ValueError: I/O operation on closed file.

In [37]:
batch = pd.read_csv(batch_path,nrows=225)

In [33]:
batch.tail()

,time,id1,id2,amount,message
220,2016-11-02 09:38:53,19633,2517,28.24,Fantasy earnings
221,2016-11-02 09:38:53,52398,8580,1.26,thanks bb
222,2016-11-02 09:38:53,26714,15402,14.25,Bojiggles
223,2016-11-02 09:38:53,65377,19063,24.47,⛽
224,2016-11-02 09:38:53,37357,3376,8.96,Alpha
